In [107]:
import re

test_set_doc = 'difficult_validate.txt'
test_set_doc_checked = test_set_doc + '.checked'

## Step 1: Load test set

In [108]:
test_items = []

with open(test_set_doc, encoding='utf-8') as infile:
    for line in infile:
        if not '\t' in line:
            continue
        test_items.append(line.strip().split('\t')[1].strip())

print(f'Loaded {len(test_items)} lines')

Loaded 310 lines


Test set annotation:
- Preverb marked by a suffixed backslash followed by an ID number, e.g. `meg\1`
- Word from which the preverb was separated marked by a pipe followed by the same ID number, e.g. `főzve|1`
- Within the same line, different verb-prefix pairs must (obviously) receive different ID numbers.
- Only single-digit ID numbers allowed.
- A preverb that is not separated from any word in the sentence (ellipsis etc.) is marked with a zero ID:
    * `"Hazakísérhetlek?" "Meg\0 hát."`
- Any number of preverbs can have the 0 ID within the same line.
- In the **difficult** dataset, a verb directly followed by preverb is only annotated if the preverb is not separated from the preceding verb:
    * `főzte meg`
    * but: `főzte|1 volna meg\1`
- In the **general (easy)** dataset, the first pattern is annotated as well:
    * `főzte|1 meg\1`
- Normally there is a 1:1 correspondence between preverbs and verbs. However, there are exceptions, and these are annotated accordingly, e.g. `Se ki\1, se be\1 nem lehetett menni|1 Budakesziről`; `át-\1 meg átjárták|1`

## Step 2: Check test set for errors and reformat for emtsv tok

In [109]:
with open(test_set_doc_checked, 'w', encoding='utf-8') as outfile:
    for t in test_items:
        prev_indices = re.findall(r'\\([1-5])', t)
        if prev_indices is None:
            print("No prevs:", t) # unannotated line
            continue
        verb_indices = re.findall(r'\|([1-5])', t)
        if (sorted(list(set(prev_indices))) != sorted(prev_indices) or # any duplicate indices
            sorted(list(set(verb_indices))) != sorted(verb_indices) or
            sorted(prev_indices) != sorted(verb_indices) # any unmatched indices
           ):
            print('Possible error:', t)
            print(prev_indices)
            print(verb_indices)
        t = re.sub(r'(\\|\|)(\d)(\S)', r'\1\2 \3', t)
        outfile.write(t.replace('\\', '_p').replace('|', '_v') + '\n')

Possible error: S így búslakodván érzetem csapkodásaiban, kedves anyám jutott eszembe, s el\1 is gondoltam|1, hogy bezzeg az ő kies kertjében más gyümölcs termik; s el\1 azt is, hogy az a gyümölcs most jobb volna nekem, mert anyám nem azt akarná, hogy én álljak helyt a bajban, hanem azt inkább, hogy az öröm állja meg mellettem a helyét.
['1', '1']
['1']


If there are errors, correct in input file, then back to step 1.

## Step 3: Tokenize in emtsv, insert testid column, run morph,pos

In [110]:
!cat {test_set_doc_checked} | docker run -i mtaril/emtsv tok > tok_output.txt

print("tok_output done")

with open('tok_output.txt', encoding='utf-8') as infile:
    with open('morph_input.txt', 'w', encoding='utf-8') as outfile:
        outfile.write(next(infile).strip() + '\ttestid\n')
        for line in infile:
            if line.strip() == '':
                outfile.write(line)
                continue
            testid = '.' # avoid removal of empty column by xtsv
            m = re.findall('_([pv]\d)', line)
            if m:
                testid = m[0]
                line = re.sub('_[pv]\d', '', line)
            outfile.write(line.strip() + '\t' + testid + '\n')

print("morph_input done")

!cat morph_input.txt | docker run -i mtaril/emtsv morph,pos > pos_output.txt

print("pos_output done")

tok_output done
morph_input done
pos_output done


In [111]:
import sys
sys.path.append('../preverb')
from word import Word

## Step 4: Check if preverbs are annotated as preverbs and verb-like tokens as verbs

In [112]:
from word import Word

with open('pos_output.txt', encoding='utf-8') as infile:
    header = next(infile)
    Word.features = header.strip().split('\t')
    for n, line in enumerate(infile):
        if line.strip() == '':
            continue
        token = Word(line.strip().split('\t'))
        if token.testid[0] == 'p' and not token.xpostag.startswith('[/Prev]'):
            print(f"Bad prev on line {n + 2}:", token, '\n')
        if token.testid[0] == 'v' and not token.xpostag.startswith('[/V]'):
            print(f"Bad verb (?) on line {n + 2}:", token, '\n')


Bad verb (?) on line 760: fogadható	" "	v1	[{"lemma": "fogadható", "tag": "[/Adj][Nom]", "morphana": "fogad[/V]=fogad+ható[_ModPtcp/Adj]=ható+[Nom]=", "readable": "fogad[/V] + ható[_ModPtcp/Adj] + [Nom]", "twolevel": "f:f o:o g:g a:a d:d :[/V] h:h a:a t:t ó:ó :[_ModPtcp/Adj] :[Nom]"}]	fogadható	[/Adj][Nom] 

Bad verb (?) on line 1146: avatkozás	" "	v1	[{"lemma": "avatkozás", "tag": "[/N][Nom]", "morphana": "avatkozik[/V]=avatkoz+ás[_Ger/N]=ás+[Nom]=", "readable": "avatkozik[/V]=avatkoz + ás[_Ger/N] + [Nom]", "twolevel": "a:a v:v a:a t:t k:k o:o z:z :i :k :[/V] á:á s:s :[_Ger/N] :[Nom]"}]	avatkozás	[/N][Nom] 

Bad verb (?) on line 1153: képzelhető	" "	v1	[{"lemma": "képzelhető", "tag": "[/Adj][Nom]", "morphana": "képzel[/V]=képzel+hető[_ModPtcp/Adj]=hető+[Nom]=", "readable": "képzel[/V] + hető[_ModPtcp/Adj] + [Nom]", "twolevel": "k:k é:é p:p z:z e:e l:l :[/V] h:h e:e t:t ő:ő :[_ModPtcp/Adj] :[Nom]"}, {"lemma": "képzelhető", "tag": "[/Adj][Nom]", "morphana": "képzelhető[/Adj]=képzelhető+


Bad verb (?) on line 6373: érhető	" "	v1	[{"lemma": "érhető", "tag": "[/Adj][Nom]", "morphana": "ér[/V]=ér+hető[_ModPtcp/Adj]=hető+[Nom]=", "readable": "ér[/V] + hető[_ModPtcp/Adj] + [Nom]", "twolevel": "é:é r:r :[/V] h:h e:e t:t ő:ő :[_ModPtcp/Adj] :[Nom]"}, {"lemma": "érhető", "tag": "[/Adj][Nom]", "morphana": "érik[/V]=ér+hető[_ModPtcp/Adj]=hető+[Nom]=", "readable": "érik[/V]=ér + hető[_ModPtcp/Adj] + [Nom]", "twolevel": "é:é r:r :i :k :[/V] h:h e:e t:t ő:ő :[_ModPtcp/Adj] :[Nom]"}]	érhető	[/Adj][Nom] 

Bad verb (?) on line 6687: osztható	" "	v2	[{"lemma": "osztható", "tag": "[/Adj][Nom]", "morphana": "oszt[/V]=oszt+ható[_ModPtcp/Adj]=ható+[Nom]=", "readable": "oszt[/V] + ható[_ModPtcp/Adj] + [Nom]", "twolevel": "o:o s:s z:z t:t :[/V] h:h a:a t:t ó:ó :[_ModPtcp/Adj] :[Nom]"}, {"lemma": "osztható", "tag": "[/Adj][Nom]", "morphana": "osztható[/Adj]=osztható+[Nom]=", "readable": "osztható[/Adj] + [Nom]", "twolevel": "o:o s:s z:z t:t h:h a:a t:t ó:ó :[/Adj] :[Nom]"}]	osztható	[/Adj][No

If there is any output, the line numbers refer to `pos_output.txt`.

Check whether tokens annotated as separated preverbs are also analysed by morph,pos as preverbs. If not (e.g. if the preverb _meg_ is tagged by emtsv as a `[/Conj]`), **remove this annotation** (or the whole item if no annotation left) from the test set because `preverb` will necessarily fail due to incorrect emtsv annotation, which is extraneous to its performance evaluation.

Exception: person-inflected preverb-like postpositions such as in `utánam\1 dobják|1`, which are tagged by emtsv as `[/Post]`, and case-inflected personal pronouns such as in `hozzá\1 voltam szokva|1`, which are tagged as `[/N|Pro]`, **should not be removed from the test set** since `preverb` should be able to handle these.

If a token is annotated as the verb stem counterpart of a separated preverb, but is not tagged by emtsv as a verb, check whether the preverb annotation is correct, but if so, **do not remove this annotation** from the test set. `preverb` is supposed to be able to handle the connection of such separated preverbs.

After all real errors have been corrected, rerun everything from step 1.

In [113]:
!cat pos_output.txt | python ../preverb > preverb_output.tsv

## Step 5: Verify diffs

NB: ONLY DO THIS ON THE VALIDATION DATASET.

In [114]:
def print_error(error_str, n, token):
    print(error_str, f"on token {n + 2}: {token.form}")

with open('preverb_output.tsv', encoding='utf-8') as infile:
    header = next(infile)
    Word.features = header.strip().split('\t')
    curr_sent = ''
    testid_to_previd = {}
    annotated_previds = {}
    irrelevant_previds = {}
    bad_flag = False
    for n, token in enumerate(infile):
        if token.strip() == '':
            if bad_flag:
                print(curr_sent, "\n")
            curr_sent = ''
            bad_flag = False
            testid_to_previd = {}
            annotated_previds = {}
            irrelevant_previds = {}
            continue
        token = Word(token.strip('\n').split('\t'))
        token_suffix = token.testid
        if token_suffix == '.':
            token_suffix = ''
        else:
            token_suffix = '_' + token_suffix
        curr_sent += token.form + token_suffix + token.wsafter.replace('"','')
        
        if token.testid == 'p0':
            if token.prev == 'conn':
                print_error("Incorrectly connected preverb", n, token)
            continue
        
        if token.testid[0] == 'p' and not token.prev == 'conn':
            print_error("Preverb mismatch", n, token)
            bad_flag = True
            continue
        elif token.testid[0] == 'v' and not token.prev == 'sep':
            print_error("Verb mismatch", n, token)
            bad_flag = True
            continue
        elif token.testid == '.' and token.previd != '':
            if token.previd in annotated_previds:
                print_error("Unannotated token connected to annotated", n, token)
                print(f"Incorrectly connected to {annotated_previds[token.previd]}")
                bad_flag = True
            else:
                irrelevant_previds[token.previd] = token.form
            continue
        elif token.previd == '':
            continue

        if token.previd in irrelevant_previds:
            print_error("Annotated token connected to unannotated", n, token)
            print(f"Incorrectly connected to {irrelevant_previds[token.previd]}")
            bad_flag = True
        elif token.testid[1] not in testid_to_previd:
            testid_to_previd[token.testid[1]] = token.previd
            annotated_previds[token.previd] = token.form
        elif testid_to_previd[token.testid[1]] != token.previd:
            print_error("Annotated token index mismatch", n, token)
            bad_flag = True

Unannotated token connected to annotated on token 295: lehetne
Incorrectly connected to el
Verb mismatch on token 299: mondani
Akkor Fehérlófia önkéntelenül magához ölelte És mintha nem is ő tenné megcsókolta Krisztina száját Sután ügyetlenül mert összekoccant a foguk Rögtön elengedte hátra_p1 is lépett_v1 tőle Zavarában el_p2 sem búcsúzott_v2 szaladt vissza_p3 se nézve_v3 Megcsókoltam ismételte ujjongva magában Megcsókoltam igen itt maradunk a hazában Együtt a hazában édes vagy édes édes\nEzt persze el_p4 lehetne nem-költői módon is mondani_v4 , de mennyivel nagyobb dolog, ha ez költői megfogalmazást kap.  

Verb mismatch on token 585: Fölidézte
Preverb mismatch on token 603: föl
Preverb mismatch on token 647: át-
Verb mismatch on token 649: átjárták
Fölidézte_v1 magában Veron jobbjának barátságos szorítását, hangjának zengését, derűs szemének biztató csillámát és jó mosolyát, föl_p1 a mély beszélgetéseket, amikor már azt hitte István, elmondott mindent, s Veron halkan kérdezni kezdet

## Step 6: Calculate evaluation metrics

In [9]:
# A debuggal sorok csak debugoláshoz / átláthatósághoz kellenek, és törölhetők, ha minden jó így.

preverb_count, true_pos, false_pos, true_neg, false_neg = 0, 0, 0, 0, 0

with open('preverb_output.tsv', encoding='utf-8') as infile:
    header = next(infile)
    Word.features = header.strip().split('\t')

    curr_sent = '' # debug

    p_testids = {}
    v_testids = {}
    irrelevant_previds = []

    for n, token in enumerate(infile):
        if token.strip() == '':
            for p_testid, p_previd in p_testids.items():
                if (p_previd in irrelevant_previds or  # \i is connected to a non-annotated verb
                    p_testid not in v_testids or       # \i is not connected to any annotated verbs
                    v_testids[p_testid] != p_previd):  # \i is not connected to |i, but some other |j
                    false_pos += 1
                    print("False pos", p_testid) # debug
                else: 
                    true_pos += 1
                    print("True pos", p_testid) # debug
            print(curr_sent)  # debug
            print("P:", p_testids) # debug
            print("V:", v_testids) # debug
            print("Irrel:", irrelevant_previds) # debug
            curr_sent = '' # debug
            p_testids = {}
            v_testids = {}
            irrelevant_previds = []
            print() # debug
            continue
        
        token = Word(token.strip('\n').split('\t'))
        token_suffix = token.testid # debug
        if token_suffix == '.': # debug
            token_suffix = '' # debug
        else: # debug
            token_suffix = '_' + token_suffix # debug
        curr_sent += token.form + token_suffix + token.wsafter.replace('"','') # debug
        
        if token.testid == '.':
            if token.previd != '':
                irrelevant_previds.append(token.previd)
            continue

        if token.testid == 'p0':
            preverb_count += 1
            if token.prev == 'conn':
                false_pos += 1   # preverb annotated as \0 connected to anything
                print("False pos") # debug
            else:
                true_neg += 1    # not connected, OK
                print("True neg") # debug
            continue
        
        if token.testid[0] == 'p':
            preverb_count += 1
            if token.prev != 'conn':
                false_neg += 1   # preverb annotated as \1 etc. should have been connected
                print("False neg", token.testid[1:]) # debug
            elif token.testid[1:] in p_testids and p_testids[token.testid[1:]] != token.previd:
                false_pos += 1   # previd does not match preverb with the same annotation
                print("False pos", token.testid[1:]) # debug
            else:
                p_testids[token.testid[1:]] = token.previd
            continue

        if token.testid[0] == 'v' and token.prev == 'sep':
            v_testids[token.testid[1:]] = token.previd        

print(preverb_count) # debug
print([true_pos, false_pos, true_neg, false_neg]) # debug
assert preverb_count == sum([true_pos, false_pos, true_neg, false_neg]) # debug

True pos 1
True pos 2
Be_p1 fog következni_v1 , mert be_p2 kell következzék_v2 az immár elkerülhetetlen totális civilizációs összeomlás.\n
P: {'1': '1', '2': '2'}
V: {'1': '1', '2': '2'}
Irrel: []

False pos 1
S le_p1 is van írva_v1 .\n
P: {'1': '3'}
V: {}
Irrel: ['3']

True pos 1
Jaj, annyi a gondom, hogy ki_p1 sem látszom_v1 belőle.\n
P: {'1': '4'}
V: {'1': '4'}
Irrel: []

True pos 1
True pos 2
Idő kérdése, de törvényszerűen alakul_v1 majd ki_p1 végül az a bizonyos új istenű és új módon értelmes világ, aminek el_p2 kell jönnie_v2 ...\n
P: {'1': '5', '2': '6'}
V: {'1': '5', '2': '6'}
Irrel: []

True pos 1
Menne ő is szívesen (miként azt később meg_p1 is teszi_v1 ), de egyelőre még szerve zési feladatai vannak.\n
P: {'1': '7'}
V: {'1': '7'}
Irrel: []

True pos 1
A nézőt valahogy meg_p1 kell tartani_v1 .\n
P: {'1': '8'}
V: {'1': '8'}
Irrel: []

True pos 1
Melyhez mintha már hozzá_p1 is szoktunk_v1 volna, mely nélkül mintha már létezni sem tudnánk.\n
P: {'1': '9'}
V: {'1': '9'}
Irrel: []

Nem is volt botrány voltaképpen, mert a fürdőzők s úszómesterek egyaránt hazamentek, csak két három elkésett látogató csavarta ki vizes fürdőruháját, megállva a medence kőszegélyén. 
P: {}
V: {}
Irrel: ['64', '64']

Mint később állították, nem hittek a saját szemüknek, megpillantva a szürkületben a meztelen alakokat. 
P: {}
V: {}
Irrel: []

Azt képzelték, érzékcsalódás áldozatai lettek, avagy talán a délibáb Egerbe költözött a Hortobágyról. 
P: {}
V: {}
Irrel: []

True pos 1
Mikor pedig rá_p1 kellett jönniük_v1 , hogy nem káprázik a szemük, lóhalálában elhagyták a fürdőt.\n
P: {'1': '65'}
V: {'1': '65'}
Irrel: []

False pos 1
True pos 2
Hol elöl, hol hátul ültek egyre többen, s a hó nőtt akkorára, hogy ha meg_p1 nem lett volna fagyva_v1 , ki_p2 sem látszik_v2 belőle az ember.\n
P: {'1': '66', '2': '67'}
V: {'2': '67'}
Irrel: ['66']

True pos 1
- Az Écel emberei be_p1 vannak építve_v1 a rendőrségre.\n
P: {'1': '68'}
V: {'1': '68'}
Irrel: []

True pos 1
Most jön_v1 csak ki_p1 a szobából 

False neg 2
True pos 1
Nehezen tudom elképzelni, hogy apám a valamelyik haszonleső maffiózó által elétárt Porhüvelyt ne vizsgálta_v1 volna meg_p1 alaposan, valóban azt kapja_v2 -e meg_p2 , amit áhít.\n
P: {'1': '139'}
V: {'1': '139'}
Irrel: []

True pos 1
Ram lelkében mozdult_v1 csak meg_p1 valami, mert ő érteni vélte a Mesternek ideáját.\n
P: {'1': '140'}
V: {'1': '140'}
Irrel: []

True pos 1
Leszúrlak, ha meg_p1 akarsz lógni_v1 .\n
P: {'1': '141'}
V: {'1': '141'}
Irrel: []

True pos 1
- Ez egy kétségbeesett segélykiáltás volt, s névvel jegyezve az első eredménye az lett volna, hogy két óra múltán már le_p1 is tartóztattak_v1 volna.\n
P: {'1': '142'}
V: {'1': '142'}
Irrel: []

True pos 1
Addig is azonban meg kellene komolyan vizsgálni, hogy azok az állami kiadók, amelyeknek elvileg magyar nyelvű könyveket is ki_p1 kellene hozniok_v1 , ezt a feladatukat miért nem teljesítik.\n
P: {'1': '143'}
V: {'1': '143'}
Irrel: []

True pos 1
Az ilyesmihez pedig hozzá_p1 voltam szokva_v1 .\n
P: {'1

False neg 1
(Marsi Péter Pál: Miért, le_p1 van zárva_v1 a vita?)\n
P: {}
V: {}
Irrel: []

True pos 1
Megvártam, amíg egy kicsit csend lesz, mert a saját szavamat nem értem, ami nem olyan nagy baj, de a kérdésnek mégiscsak el_p1 kell hangozni_v1 .\n
P: {'1': '207'}
V: {'1': '207'}
Irrel: []

False neg 1
Ha azt mondom, hogy 2-300 milliárd beruházásigényről van szó, és ehhez képest 3 milliárd kerül_v1 ide be_p1 , akkor azt kell mondjam, hogy az aránytalan, ez így hiba.\n
P: {}
V: {}
Irrel: []

False neg 1
True neg
True neg
True pos 1
Az elnyert támogatás kizárólag a tárgyévben meghirdetett tervpályázatok céljára használható_v1 fel_p1 .\n* Vissza_p0 a tartalomjegyzékhez\nÚ vissza_p0 a címlaphoz\n(1) Minden sírhelytábla számát jelzőoszlopon, a sírhelysorok és sírhelyek számát pedig a sorokon túl jól látható helyen fel_p1 kell tüntetni_v1 .\n
P: {'1': '208'}
V: {'1': '208'}
Irrel: []

False neg 1
True pos 1
Az erőmű építés vonatkozásában megvizsgálandó annak lehetősége is, hogy a beruházás t

True pos 1
Akik (Anglia) elöre tudták, hogy a németek bombázni fogják Coventryt, s hagyták lebombázni a polgári lakosságot, csak hogy ki_p1 ne derüljön_v1 : megfejtették a Luftwaffe kódrendszerét (mert ha kiürítik a várost, egyértelművé válik a megfejtés ténye)?\n
P: {'1': '269'}
V: {'1': '269'}
Irrel: []

True pos 1
Pl. hogy ki szegte meg szavát: el_p1 ne feledd_v1 tisztázni.\n
P: {'1': '271'}
V: {'1': '271'}
Irrel: ['270', '270']

True pos 1
Mégpedig azért, mert nyakunkon a tavasz meg a nyár, és - csak el_p1 ne kiabáljam_v1 - mintha lanyhulna a melltartó-terror.\n
P: {'1': '272'}
V: {'1': '272'}
Irrel: []

True pos 1
A múltkor kedves vizslámat elütötte egy autó, az ölemben " mondott " búcsút a közös sétákért, emlékekért és kölcsönös szeretetért, Akkor sírtam, mert ez volt a valóság és el_p1 kellett fogadni_v1 , hogy nincs tovább!\n
P: {'1': '273'}
V: {'1': '273'}
Irrel: []

True pos 1
Ős patkány terjeszt kórt miköztünk, a meg_p1 nem gondolt_v1 gondolat, belezabál, amit kifőztünk, s e

True pos 1
Egyszer egy zsaru éjfél fele alig akarta elhinni, hogy be_p1 van téve_v1 a kontaktlencsém. 
P: {'1': '339'}
V: {'1': '339'}
Irrel: []

True pos 2
El_p2 kellett olvasnom_v2 egy hirdetőtábla szövegét.\n
P: {'2': '340'}
V: {'2': '340'}
Irrel: []

True pos 1
Lehet, hogy mégis csak be_p1 kéne ruházni_v1 egy Uziba?!\n
P: {'1': '341'}
V: {'1': '341'}
Irrel: []

True pos 1
Amikor belekezdett a felgöngyölítősdibe, akkor számolnia kellett a következményeivel, hogy sokan ki_p1 fognak akadni_v1 miatta.\n
P: {'1': '342'}
V: {'1': '342'}
Irrel: []

False neg 1
Samu a jól nevelt Ír szetter be_p1 kászálódott_v1 az ülésre, s mint aki 1 hete nem evett széles nyelvcsapásokkal lenyalta az aznapi bűnjelet.\n
P: {}
V: {}
Irrel: []

True pos 1
Szerintem ha így folytatódik, lassan az lesz a " jó " flim pl a tévében, amelyiket nem szakítja meg reklám, s a végén a " sötétig " láthatjuk a kockákat, hiszen a jobb film mellé kevesebb reklámot is el_p1 lehet adni_v1 ugyanannyiért, mint amennyit a sorozat

In [10]:
# precision:
precision = true_pos / (true_pos + false_pos)
print("Precision: %.4f"%precision)

# recall:
recall = true_pos / (true_pos + false_neg)
print("Recall: %.4f"%recall)

# F1
print("F1: %.4f"%(2 * precision * recall / (precision + recall)))

# accuracy:
print("Accuracy: %.4f"%((true_pos + true_neg) / preverb_count))

Precision: 0.9681
Recall: 0.7982
F1: 0.8750
Accuracy: 0.7821


## Step 7: Evaluate the baseline algorithms

In [11]:
!cat preverb_output.tsv | python old_connect_prev.py > baseline_output.tsv

In [12]:
# Basically same procedure as above, debug lines removed, wrapped in a function,
# abstraction over target field name

PREVERB_COUNT, TRUE_POS, FALSE_POS, TRUE_NEG, FALSE_NEG = 0, 1, 2, 3, 4

def evaluate_target_column(input_file_name, target_col_name):
    preverb_count, true_pos, false_pos, true_neg, false_neg = 0, 0, 0, 0, 0

    with open(input_file_name, encoding='utf-8') as infile:
        header = next(infile)
        Word.features = header.strip().split('\t')

        p_testids = {}
        v_testids = {}
        irrelevant_previds = []

        for n, token in enumerate(infile):
            if token.strip() == '':
                for p_testid, p_target_id in p_testids.items():
                    if (p_target_id in irrelevant_previds or  # \i is connected to a non-annotated verb
                        p_testid not in v_testids or          # \i is not connected to any annotated verbs
                        v_testids[p_testid] != p_target_id):  # \i is not connected to |i, but some other |j
                        false_pos += 1
                    else: 
                        true_pos += 1
                p_testids = {}
                v_testids = {}
                irrelevant_previds = []
                continue

            token = Word(token.strip('\n').split('\t'))
            target_id = getattr(token, target_col_name)
            
            if token.testid == '.':
                if target_id not in ('.', ''):
                    irrelevant_previds.append(target_id)
                continue

            if token.testid == 'p0':
                preverb_count += 1
                if target_id in ('.',''):
                    true_neg += 1    # not connected, OK
                else:
                    false_pos += 1   # preverb annotated as \0 connected to anything
                continue

            if token.testid[0] == 'p':
                preverb_count += 1
                if target_id in ('.',''):
                    false_neg += 1   # preverb annotated as \1 etc. should have been connected
                elif token.testid[1:] in p_testids and p_testids[token.testid[1:]] != target_id:
                    false_pos += 1   # previd does not match preverb with the same annotation
                else:
                    p_testids[token.testid[1:]] = target_id
                continue

            if token.testid[0] == 'v' and target_id not in ('.',''):
                v_testids[token.testid[1:]] = target_id
    
    return (preverb_count, true_pos, false_pos, true_neg, false_neg)

def calculate_metrics(results):
    # precision:
    precision = results[TRUE_POS] / (results[TRUE_POS] + results[FALSE_POS])
    print("Precision: %.4f"%precision)

    # recall:
    recall = results[TRUE_POS] / (results[TRUE_POS] + results[FALSE_NEG])
    print("Recall: %.4f"%recall)

    # F1
    print("F1: %.4f"%(2 * precision * recall / (precision + recall)))

    # accuracy:
    print("Accuracy: %.4f"%((results[TRUE_POS] + results[TRUE_NEG]) / results[PREVERB_COUNT]))

print("Evaluate old connect_prev baseline")

results = evaluate_target_column('baseline_output.tsv', 'prevold')
print(results)
assert results[PREVERB_COUNT] == sum([results[TRUE_POS], 
                                      results[FALSE_POS],
                                      results[TRUE_NEG],
                                      results[FALSE_NEG]]) # debug
calculate_metrics(results)
print()

print("Evaluate connect closest baseline")
results = evaluate_target_column('baseline_output.tsv', 'prevclosest')
print(results)
assert results[PREVERB_COUNT] == sum([results[TRUE_POS], 
                                      results[FALSE_POS],
                                      results[TRUE_NEG],
                                      results[FALSE_NEG]]) # debug
calculate_metrics(results)
print()


Evaluate old connect_prev baseline
(358, 282, 15, 7, 54)
Precision: 0.9495
Recall: 0.8393
F1: 0.8910
Accuracy: 0.8073

Evaluate connect closest baseline
(358, 269, 77, 2, 10)
Precision: 0.7775
Recall: 0.9642
F1: 0.8608
Accuracy: 0.7570



## Step 8: Evaluate Stanza preverb connections

In [13]:
!cat difficult_validate.txt | python reformat_for_tok.py | docker run -i mtaril/emtsv emstanza-tok > stanzatok_output.txt
!cat stanzatok_output.txt | python label_to_column.py | \
    docker run -i mtaril/emtsv emstanza-lem | \
    docker run -i mtaril/emtsv emstanza-parse > stanzaparse_output.txt

In [14]:
from more_itertools import split_at

with open('stanzaparse_output.txt', encoding='utf-8') as infile:
    with open('stanzaprev.txt', 'w', encoding='utf-8') as outfile:
        header = next(infile)
        Word.features = header.strip().split('\t') + ['stanzaid']
        outfile.write(Word.header() + '\n')
        lines = infile.read().split('\n')
        sentences = list(split_at(lines, lambda x: x == ''))
        for s in sentences:
            words = [Word(token.split('\t') + ['.']) for token in s]
            for w in words:
                if w.deprel == 'compound:preverb':
                    w.stanzaid = w.head
                    words[int(w.head) - 1].stanzaid = w.head
            for w in words:
                outfile.write(str(w) + '\n')
            outfile.write('\n')

In [15]:
results = evaluate_target_column("stanzaprev.txt", 'stanzaid')
print(results)
assert results[PREVERB_COUNT] == sum([results[TRUE_POS], 
                                      results[FALSE_POS],
                                      results[TRUE_NEG],
                                      results[FALSE_NEG]]) # debug
calculate_metrics(results)
print()

(358, 291, 28, 0, 39)
Precision: 0.9122
Recall: 0.8818
F1: 0.8968
Accuracy: 0.8128



## Step 9: Generalising evaluation workflow

In [118]:
test_set_name = 'difficult_validate'
test_set_raw = test_set_name + '.txt'
test_set_reformatted = test_set_name + '_reformat.txt'
test_set_emtok_output = test_set_name + '_emtok_output.tsv'
test_set_empos_output = test_set_name + '_empos_output.tsv'
test_set_preverb_output = test_set_name + '_preverb_output.tsv'
test_set_baseline_output = test_set_name + '_baseline_output.tsv'

test_set_stanza_tok_output = test_set_name + '_stanzatok_output.tsv'
test_set_stanza_parse_output = test_set_name + '_stanzaparse_output.tsv'
test_set_stanza_ids = test_set_name + '_stanzaids.tsv'

In [119]:
!cat {test_set_raw} | python reformat_for_tok.py > {test_set_reformatted}
!cat {test_set_reformatted} | docker run -i mtaril/emtsv tok > {test_set_emtok_output}
!cat {test_set_emtok_output} | python label_to_column.py | docker run -i mtaril/emtsv morph,pos > {test_set_empos_output}
!cat {test_set_empos_output} | python ../preverb > {test_set_preverb_output}
!cat {test_set_preverb_output} | python old_connect_prev.py > {test_set_baseline_output}

In [120]:
!cat {test_set_reformatted} | docker run -i mtaril/emtsv emstanza-tok > {test_set_stanza_tok_output}
!cat {test_set_stanza_tok_output} | python label_to_column.py | \
    docker run -i mtaril/emtsv emstanza-lem | \
    docker run -i mtaril/emtsv emstanza-parse > {test_set_stanza_parse_output}
!cat {test_set_stanza_parse_output} | python add_stanzaid.py > {test_set_stanza_ids}

In [121]:
print("emPreverb results")
results = evaluate_target_column(test_set_baseline_output, 'previd')
calculate_metrics(results)
print()

# initial results:
# emPreverb results
# Precision: 0.9823
# Recall: 0.8006
# F1: 0.8822
# Accuracy: 0.7933

# previous results:
# emPreverb results
# Precision: 0.9842
# Recall: 0.9043
# F1: 0.9426
# Accuracy: 0.8936
    
print("old connect_prev (max2) results")
results = evaluate_target_column(test_set_baseline_output, 'prevold')
calculate_metrics(results)
print()

print("closest verb baseline results")
results = evaluate_target_column(test_set_baseline_output, 'prevclosest')
calculate_metrics(results)
print()

print("stanza results")
results = evaluate_target_column(test_set_stanza_ids, 'stanzaid')
calculate_metrics(results)
print()

emPreverb results
Precision: 0.9912
Recall: 0.9684
F1: 0.9797
Accuracy: 0.9608

old connect_prev (max2) results
Precision: 0.9628
Recall: 0.8407
F1: 0.8976
Accuracy: 0.8179

closest verb baseline results
Precision: 0.7942
Recall: 0.9648
F1: 0.8712
Accuracy: 0.7731

stanza results
Precision: 0.9119
Recall: 0.8815
F1: 0.8964
Accuracy: 0.8123



## Step 10: Evaluation on general (easy) test dataset

In [122]:
test_set_name = 'gold500_test'
test_set_raw = test_set_name + '.txt'
test_set_reformatted = test_set_name + '_reformat.txt'
test_set_emtok_output = test_set_name + '_emtok_output.tsv'
test_set_empos_output = test_set_name + '_empos_output.tsv'
test_set_preverb_output = test_set_name + '_preverb_output.tsv'
test_set_baseline_output = test_set_name + '_baseline_output.tsv'

test_set_stanza_tok_output = test_set_name + '_stanzatok_output.tsv'
test_set_stanza_parse_output = test_set_name + '_stanzaparse_output.tsv'
test_set_stanza_ids = test_set_name + '_stanzaids.tsv'

!cat {test_set_raw} | python reformat_for_tok.py > {test_set_reformatted}
!cat {test_set_reformatted} | docker run -i mtaril/emtsv tok > {test_set_emtok_output}
!cat {test_set_emtok_output} | python label_to_column.py | docker run -i mtaril/emtsv morph,pos > {test_set_empos_output}
!cat {test_set_empos_output} | python ../preverb > {test_set_preverb_output}
!cat {test_set_preverb_output} | python old_connect_prev.py > {test_set_baseline_output}

!cat {test_set_reformatted} | docker run -i mtaril/emtsv emstanza-tok > {test_set_stanza_tok_output}
!cat {test_set_stanza_tok_output} | python label_to_column.py | \
    docker run -i mtaril/emtsv emstanza-lem | \
    docker run -i mtaril/emtsv emstanza-parse > {test_set_stanza_parse_output}
!cat {test_set_stanza_parse_output} | python add_stanzaid.py > {test_set_stanza_ids}

In [123]:
print("emPreverb results")
results = evaluate_target_column(test_set_baseline_output, 'previd')
calculate_metrics(results)
print()

print("old connect_prev (max2) results")
results = evaluate_target_column(test_set_baseline_output, 'prevold')
calculate_metrics(results)
print()

print("closest verb baseline results")
results = evaluate_target_column(test_set_baseline_output, 'prevclosest')
calculate_metrics(results)
print()

print("stanza results")
results = evaluate_target_column(test_set_stanza_ids, 'stanzaid')
calculate_metrics(results)
print()

emPreverb results
Precision: 0.9980
Recall: 0.9780
F1: 0.9879
Accuracy: 0.9760

old connect_prev (max2) results
Precision: 0.9794
Recall: 0.9694
F1: 0.9744
Accuracy: 0.9500

closest verb baseline results
Precision: 0.9656
Recall: 0.9876
F1: 0.9765
Accuracy: 0.9540

stanza results
Precision: 0.9604
Recall: 0.9066
F1: 0.9328
Accuracy: 0.8740



## Step 11: Evaluation on difficult test dataset

In [124]:
test_set_name = 'difficult_test'
test_set_raw = test_set_name + '.txt'
test_set_reformatted = test_set_name + '_reformat.txt'
test_set_emtok_output = test_set_name + '_emtok_output.tsv'
test_set_empos_output = test_set_name + '_empos_output.tsv'
test_set_preverb_output = test_set_name + '_preverb_output.tsv'
test_set_baseline_output = test_set_name + '_baseline_output.tsv'

test_set_stanza_tok_output = test_set_name + '_stanzatok_output.tsv'
test_set_stanza_parse_output = test_set_name + '_stanzaparse_output.tsv'
test_set_stanza_ids = test_set_name + '_stanzaids.tsv'

!cat {test_set_raw} | python reformat_for_tok.py > {test_set_reformatted}
!cat {test_set_reformatted} | docker run -i mtaril/emtsv tok > {test_set_emtok_output}
!cat {test_set_emtok_output} | python label_to_column.py | docker run -i mtaril/emtsv morph,pos > {test_set_empos_output}
!cat {test_set_empos_output} | python ../preverb > {test_set_preverb_output}
!cat {test_set_preverb_output} | python old_connect_prev.py > {test_set_baseline_output}

!cat {test_set_reformatted} | docker run -i mtaril/emtsv emstanza-tok > {test_set_stanza_tok_output}
!cat {test_set_stanza_tok_output} | python label_to_column.py | \
    docker run -i mtaril/emtsv emstanza-lem | \
    docker run -i mtaril/emtsv emstanza-parse > {test_set_stanza_parse_output}
!cat {test_set_stanza_parse_output} | python add_stanzaid.py > {test_set_stanza_ids}

In [125]:
print("emPreverb results")
results = evaluate_target_column(test_set_baseline_output, 'previd')
calculate_metrics(results)
print()

print("old connect_prev (max2) results")
results = evaluate_target_column(test_set_baseline_output, 'prevold')
calculate_metrics(results)
print()

print("closest verb baseline results")
results = evaluate_target_column(test_set_baseline_output, 'prevclosest')
calculate_metrics(results)
print()

print("stanza results")
results = evaluate_target_column(test_set_stanza_ids, 'stanzaid')
calculate_metrics(results)
print()

emPreverb results
Precision: 0.9912
Recall: 0.9210
F1: 0.9548
Accuracy: 0.9149

old connect_prev (max2) results
Precision: 0.9228
Recall: 0.7948
F1: 0.8540
Accuracy: 0.7500

closest verb baseline results
Precision: 0.7662
Recall: 0.9379
F1: 0.8434
Accuracy: 0.7314

stanza results
Precision: 0.9083
Recall: 0.8924
F1: 0.9003
Accuracy: 0.8191

